In [3]:
import json
import pickle
import numpy as np
import pandas as pd
from sklearn.cluster import AgglomerativeClustering
from sklearn.neighbors import kneighbors_graph

## 272

In [4]:
df = pd.read_csv('272-predictions-with-labels.csv')
admissions = pd.read_csv('AttentionHCare/data/ADMISSIONS.csv')
diagnoses = pd.read_csv('AttentionHCare/data/DIAGNOSES_ICD.csv')
patients = pd.read_csv('AttentionHCare/data/PATIENTS.csv')
df

,Essential hypertension,Cardiac dysrhythmias,Heart failure,Disorders of fluid electrolyte and acid-base balance,Diabetes mellitus,Other and unspecified anemias,Other diseases of lung,Other forms of chronic ischemic heart disease,Disorders of lipoid metabolism,Acute kidney failure,...,Foreign body in pharynx and larynx,Myoneural disorders,Personality disorders,Other congenital anomalies of circulatory system,Other congenital anomalies of heart,Family history of certain other specific conditions,Other and ill-defined conditions originating in the perinatal period,Late effects of injuries to the nervous system,label,subject_id
0,0.004109,0.004989,0.028769,0.012968,0.008394,0.007785,0.050617,0.018794,0.024205,0.022498,...,0.000124,0.000124,0.000128,0.000124,0.000124,0.000124,0.000124,0.000124,1,28410.0
1,0.016219,0.020661,0.032814,0.064551,0.024989,0.029795,0.017165,0.018680,0.018489,0.012512,...,0.000217,0.000217,0.000225,0.000217,0.000217,0.000217,0.000217,0.000217,1,27901.0
2,0.068355,0.067206,0.031333,0.063707,0.023026,0.033221,0.019513,0.021010,0.015112,0.020805,...,0.000092,0.000092,0.000094,0.000092,0.000092,0.000092,0.000092,0.000092,9,9838.0
3,0.007166,0.006288,0.025205,0.009711,0.026280,0.023621,0.009569,0.059926,0.012978,0.010989,...,0.000315,0.000315,0.000327,0.000315,0.000315,0.000315,0.000315,0.000315,11,28775.0
4,0.007350,0.014307,0.036147,0.022450,0.023340,0.018980,0.019395,0.017671,0.020393,0.009504,...,0.000176,0.000176,0.000182,0.000176,0.000176,0.000176,0.000176,0.000176,8,29570.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7477,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,13,20643.0
7478,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,13,5060.0
7479,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,13,11861.0
7480,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,13,109.0


In [5]:
df['subject_id'] = pd.to_numeric(df['subject_id'], downcast='signed')
df = df.reset_index(drop=True)
df = df.rename(columns={0: 'subject_id'})
col = df.pop("subject_id")
df.insert(0, col.name, col)
df

,subject_id,Essential hypertension,Cardiac dysrhythmias,Heart failure,Disorders of fluid electrolyte and acid-base balance,Diabetes mellitus,Other and unspecified anemias,Other diseases of lung,Other forms of chronic ischemic heart disease,Disorders of lipoid metabolism,...,Acquired hemolytic anemias,Foreign body in pharynx and larynx,Myoneural disorders,Personality disorders,Other congenital anomalies of circulatory system,Other congenital anomalies of heart,Family history of certain other specific conditions,Other and ill-defined conditions originating in the perinatal period,Late effects of injuries to the nervous system,label
0,28410,0.004109,0.004989,0.028769,0.012968,0.008394,0.007785,0.050617,0.018794,0.024205,...,0.000124,0.000124,0.000124,0.000128,0.000124,0.000124,0.000124,0.000124,0.000124,1
1,27901,0.016219,0.020661,0.032814,0.064551,0.024989,0.029795,0.017165,0.018680,0.018489,...,0.000217,0.000217,0.000217,0.000225,0.000217,0.000217,0.000217,0.000217,0.000217,1
2,9838,0.068355,0.067206,0.031333,0.063707,0.023026,0.033221,0.019513,0.021010,0.015112,...,0.000092,0.000092,0.000092,0.000094,0.000092,0.000092,0.000092,0.000092,0.000092,9
3,28775,0.007166,0.006288,0.025205,0.009711,0.026280,0.023621,0.009569,0.059926,0.012978,...,0.000315,0.000315,0.000315,0.000327,0.000315,0.000315,0.000315,0.000315,0.000315,11
4,29570,0.007350,0.014307,0.036147,0.022450,0.023340,0.018980,0.019395,0.017671,0.020393,...,0.000176,0.000176,0.000176,0.000182,0.000176,0.000176,0.000176,0.000176,0.000176,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7477,20643,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,13
7478,5060,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,13
7479,11861,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,13
7480,109,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,13


In [6]:
# df = pd.read_csv('theano-to-tf/272-attentionhcare-271-codes_prediction.csv', header=None)
# df

In [7]:
# # Clustering
# knn_graph = kneighbors_graph(df.iloc[:, 1:], 20, include_self=False)
# labels = AgglomerativeClustering(n_clusters=14, connectivity=knn_graph).fit_predict(df.iloc[:, 1:])
# df['label'] = labels.T

### Admissions

In [8]:
demographics_features = ["subject_id", "admission_type", "admission_location", "discharge_location", "insurance", "language", "religion", "marital_status", "ethnicity"]
admissions_demographics = admissions[demographics_features]
admissions_demographics.head()

,subject_id,admission_type,admission_location,discharge_location,insurance,language,religion,marital_status,ethnicity
0,2,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,NaN,NOT SPECIFIED,NaN,ASIAN
1,3,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,NaN,CATHOLIC,MARRIED,WHITE
2,4,EMERGENCY,EMERGENCY ROOM ADMIT,HOME WITH HOME IV PROVIDR,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE
3,5,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,NaN,BUDDHIST,NaN,ASIAN
4,6,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE


In [9]:
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_graphviz
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import balanced_accuracy_score, accuracy_score, precision_score, recall_score, f1_score
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.naive_bayes import GaussianNB

subject_df = df[["subject_id", "label"]]
merged_df = pd.DataFrame.merge(subject_df, admissions_demographics, on='subject_id', validate='one_to_many')
cols = merged_df.columns.tolist()
cols.insert(len(merged_df.columns.tolist()), cols.pop(cols.index('label')))
merged_df = merged_df.reindex(columns=cols)
merged_df = merged_df.apply(lambda x: pd.factorize(x)[0])

for col in cols[1:]:
    merged_df[col] = merged_df[col].astype('category')

X, y = merged_df.iloc[:, 1:-1], merged_df.iloc[:, [-1]].values

X.info()
X.head()

accuracy_tree = []
precision_tree = []
recall_tree = []
f1_score_tree = []

accuracy_dummy = []
precision_dummy = []
recall_dummy = []
f1_score_dummy = []

accuracy_nb = []
precision_nb = []
recall_nb = []
f1_score_nb = []


n_splits = 5
# model = OneVsRestClassifier(DecisionTreeClassifier(class_weight="balanced"))
model = DecisionTreeClassifier(class_weight="balanced", min_samples_split=50, min_samples_leaf=50)

# dummy_model = OneVsRestClassifier(DummyClassifier(strategy="most_frequent"))
dummy_model = DummyClassifier(strategy="most_frequent")
skf = StratifiedKFold(n_splits=n_splits, random_state=0, shuffle=True)

nb_model = GaussianNB()

for train_index, test_index in skf.split(X, y):
    # Separa em variáveis de treino e teste
    x_train, y_train = X.values[train_index], y[train_index]
    x_test, y_test = X.values[test_index], y[test_index]

    # Fit do classificador c
    fitted_model = model.fit(x_train, y_train.ravel())
    fitted_dummy = dummy_model.fit(x_train, y_train.ravel())
    fitted_nb = nb_model.fit(x_train, y_train.ravel())

    # Predição dos valores de teste e valor de acurácia
    y_pred = fitted_model.predict(x_test)
    y_dummy = fitted_dummy.predict(x_test)
    y_nb = fitted_nb.predict(x_test)
    
    accuracy_tree.append(accuracy_score(y_test, y_pred))
    precision_tree.append(precision_score(y_test, y_pred, average='weighted'))
    recall_tree.append(recall_score(y_test, y_pred, average='weighted'))
    f1_score_tree.append(f1_score(y_test, y_pred, average='weighted'))

    accuracy_dummy.append(accuracy_score(y_test, y_dummy))
    precision_dummy.append(precision_score(y_test, y_dummy, average='weighted'))
    recall_dummy.append(recall_score(y_test, y_dummy, average='weighted'))
    f1_score_dummy.append(f1_score(y_test, y_dummy, average='weighted'))

    accuracy_nb.append(accuracy_score(y_test, y_nb))
    precision_nb.append(precision_score(y_test, y_nb, average='weighted'))
    recall_nb.append(recall_score(y_test, y_nb, average='weighted'))
    f1_score_nb.append(f1_score(y_test, y_nb, average='weighted'))

    
# importance = model.feature_importances_
# for i,v in enumerate(zip(importance, demographics_features[1:])):
#     print(i, v)
#     print(demographics_features[i+1])
#     print('Score: %.5f' % (v))

print(np.mean(accuracy_tree))
print(np.mean(precision_tree))
print(np.mean(recall_tree))
print(np.mean(f1_score_tree))
print("")
print(np.mean(accuracy_dummy))
print(np.mean(precision_dummy))
print(np.mean(recall_dummy))
print(np.mean(f1_score_dummy))
print("")
print(np.mean(accuracy_nb))
print(np.mean(precision_nb))
print(np.mean(recall_nb))
print(np.mean(f1_score_nb))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19883 entries, 0 to 19882
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   admission_type      19883 non-null  category
 1   admission_location  19883 non-null  category
 2   discharge_location  19883 non-null  category
 3   insurance           19883 non-null  category
 4   language            19883 non-null  category
 5   religion            19883 non-null  category
 6   marital_status      19883 non-null  category
 7   ethnicity           19883 non-null  category
dtypes: category(8)
memory usage: 316.5 KB


/usr/local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


NameError: name 'accuracy_nb' is not defined

In [ ]:
export_graphviz(model, out_file="272/272-admissions.dot", feature_names=list(X.columns), filled=True, rounded=True, impurity=False, class_names=model.classes_.astype(str))


### ICD + Admissions

In [ ]:
admissions_demographics = admissions_demographics.apply(lambda x: pd.factorize(x)[0])
pred_demographics_df = pd.DataFrame.merge(df, admissions_demographics, on='subject_id', validate='one_to_many')

cols = pred_demographics_df.columns.tolist()
cols.insert(len(pred_demographics_df.columns.tolist()), cols.pop(cols.index('label')))
pred_demographics_df = pred_demographics_df.reindex(columns=cols)

for col in cols[1:]:
    pred_demographics_df[col] = pred_demographics_df[col].astype('category')

In [ ]:
pred_demographics_df

In [ ]:
X, y = pred_demographics_df.iloc[:, 1:-1], pred_demographics_df.iloc[:, [-1]].values

X.info()
X.head()

accuracy_tree = []
precision_tree = []
recall_tree = []
f1_score_tree = []

accuracy_dummy = []
precision_dummy = []
recall_dummy = []
f1_score_dummy = []

accuracy_nb = []
precision_nb = []
recall_nb = []
f1_score_nb = []


n_splits = 5
# model = OneVsRestClassifier(DecisionTreeClassifier(class_weight="balanced"))
model = DecisionTreeClassifier(class_weight="balanced", min_samples_split=50, min_samples_leaf=50)

# dummy_model = OneVsRestClassifier(DummyClassifier(strategy="most_frequent"))
dummy_model = DummyClassifier(strategy="most_frequent")

nb_model = GaussianNB()

skf = StratifiedKFold(n_splits=n_splits, random_state=0, shuffle=True)

for train_index, test_index in skf.split(X, y):
    # Separa em variáveis de treino e teste
    x_train, y_train = X.values[train_index], y[train_index]
    x_test, y_test = X.values[test_index], y[test_index]

    # Fit do classificador c
    fitted_model = model.fit(x_train, y_train.ravel())
    fitted_dummy = dummy_model.fit(x_train, y_train.ravel())
    fitted_nb = nb_model.fit(x_train, y_train.ravel())

    # Predição dos valores de teste e valor de acurácia
    y_pred = fitted_model.predict(x_test)
    y_dummy = fitted_dummy.predict(x_test)
    y_nb = fitted_nb.predict(x_test)
    
    accuracy_tree.append(accuracy_score(y_test, y_pred))
    precision_tree.append(precision_score(y_test, y_pred, average='weighted'))
    recall_tree.append(recall_score(y_test, y_pred, average='weighted'))
    f1_score_tree.append(f1_score(y_test, y_pred, average='weighted'))

    accuracy_dummy.append(accuracy_score(y_test, y_dummy))
    precision_dummy.append(precision_score(y_test, y_dummy, average='weighted'))
    recall_dummy.append(recall_score(y_test, y_dummy, average='weighted'))
    f1_score_dummy.append(f1_score(y_test, y_dummy, average='weighted'))

    accuracy_nb.append(accuracy_score(y_test, y_nb))
    precision_nb.append(precision_score(y_test, y_nb, average='weighted'))
    recall_nb.append(recall_score(y_test, y_nb, average='weighted'))
    f1_score_nb.append(f1_score(y_test, y_nb, average='weighted'))

# importance = model.feature_importances_
# for i,v in enumerate(zip(importance, pred_demographics_df[1:])):
#     print(i, v)

print(np.mean(accuracy_tree))
print(np.mean(precision_tree))
print(np.mean(recall_tree))
print(np.mean(f1_score_tree))
print("")
print(np.mean(accuracy_dummy))
print(np.mean(precision_dummy))
print(np.mean(recall_dummy))
print(np.mean(f1_score_dummy))
print("")
print(np.mean(accuracy_nb))
print(np.mean(precision_nb))
print(np.mean(recall_nb))
print(np.mean(f1_score_nb))

In [ ]:
export_graphviz(model, out_file="272/272-icd_admissions.dot", feature_names=list(X.columns), filled=True, rounded=True, impurity=False, class_names=model.classes_.astype(str))


### Admissions + Patients

In [ ]:
patients_codes = patients[["SUBJECT_ID", "GENDER", "DOB", "DOD", "DOD_HOSP", "DOD_SSN"]]
admissions_patients = pd.DataFrame.merge(admissions_demographics, patients_codes, left_on='subject_id', right_on='SUBJECT_ID', validate='many_to_many')
admissions_patients = admissions_patients.drop('SUBJECT_ID', axis=1)
admissions_patients = admissions_patients.apply(lambda x: pd.factorize(x)[0])

In [ ]:
subject_df = df[["subject_id", "label"]]
merged_df = pd.DataFrame.merge(subject_df, admissions_patients, on='subject_id', validate='one_to_many')
cols = merged_df.columns.tolist()
cols.insert(len(merged_df.columns.tolist()), cols.pop(cols.index('label')))
merged_df = merged_df.reindex(columns=cols)
merged_df = merged_df.apply(lambda x: pd.factorize(x)[0])

for col in cols[1:]:
    merged_df[col] = merged_df[col].astype('category')

X, y = merged_df.iloc[:, 1:-1], merged_df.iloc[:, [-1]].values

X.info()
X.head()

accuracy_tree = []
precision_tree = []
recall_tree = []
f1_score_tree = []

accuracy_dummy = []
precision_dummy = []
recall_dummy = []
f1_score_dummy = []

accuracy_nb = []
precision_nb = []
recall_nb = []
f1_score_nb = []


n_splits = 5
# model = OneVsRestClassifier(DecisionTreeClassifier(class_weight="balanced"))
model = DecisionTreeClassifier(class_weight="balanced", min_samples_split=50, min_samples_leaf=50)

# dummy_model = OneVsRestClassifier(DummyClassifier(strategy="most_frequent"))

nb_model = GaussianNB()

dummy_model = DummyClassifier(strategy="most_frequent")

skf = StratifiedKFold(n_splits=n_splits, random_state=0, shuffle=True)

for train_index, test_index in skf.split(X, y):
    # Separa em variáveis de treino e teste
    x_train, y_train = X.values[train_index], y[train_index]
    x_test, y_test = X.values[test_index], y[test_index]

    # Fit do classificador c
    fitted_model = model.fit(x_train, y_train.ravel())
    fitted_dummy = dummy_model.fit(x_train, y_train.ravel())
    fitted_nb = nb_model.fit(x_train, y_train.ravel())

    # Predição dos valores de teste e valor de acurácia
    y_pred = fitted_model.predict(x_test)
    y_dummy = fitted_dummy.predict(x_test)
    y_nb = fitted_nb.predict(x_test)

    accuracy_tree.append(accuracy_score(y_test, y_pred))
    precision_tree.append(precision_score(y_test, y_pred, average='weighted'))
    recall_tree.append(recall_score(y_test, y_pred, average='weighted'))
    f1_score_tree.append(f1_score(y_test, y_pred, average='weighted'))

    accuracy_dummy.append(accuracy_score(y_test, y_dummy))
    precision_dummy.append(precision_score(y_test, y_dummy, average='weighted'))
    recall_dummy.append(recall_score(y_test, y_dummy, average='weighted'))
    f1_score_dummy.append(f1_score(y_test, y_dummy, average='weighted'))

    accuracy_nb.append(accuracy_score(y_test, y_nb))
    precision_nb.append(precision_score(y_test, y_nb, average='weighted'))
    recall_nb.append(recall_score(y_test, y_nb, average='weighted'))
    f1_score_nb.append(f1_score(y_test, y_nb, average='weighted'))

# importance = model.feature_importances_
# for i,v in enumerate(zip(importance, pred_demographics_df[1:])):
#     print(i, v)

print(np.mean(accuracy_tree))
print(np.mean(precision_tree))
print(np.mean(recall_tree))
print(np.mean(f1_score_tree))
print("")
print(np.mean(accuracy_dummy))
print(np.mean(precision_dummy))
print(np.mean(recall_dummy))
print(np.mean(f1_score_dummy))
print("")
print(np.mean(accuracy_nb))
print(np.mean(precision_nb))
print(np.mean(recall_nb))
print(np.mean(f1_score_nb))

In [ ]:
export_graphviz(model, out_file="272/272-admissions_patients.dot", feature_names=list(X.columns), filled=True, rounded=True, impurity=False, class_names=model.classes_.astype(str))


### ICD + Admissions + Patients

In [ ]:
admissions_patients = admissions_patients.apply(lambda x: pd.factorize(x)[0])
pred_patients_demographics_df = pd.DataFrame.merge(df, admissions_patients, on='subject_id', validate='one_to_many')

cols = pred_patients_demographics_df.columns.tolist()
cols.insert(len(pred_patients_demographics_df.columns.tolist()), cols.pop(cols.index('label')))
pred_patients_demographics_df = pred_patients_demographics_df.reindex(columns=cols)

for col in cols[1:]:
    pred_patients_demographics_df[col] = pred_patients_demographics_df[col].astype('category')

In [ ]:
accuracy_tree = []
precision_tree = []
recall_tree = []
f1_score_tree = []

accuracy_dummy = []
precision_dummy = []
recall_dummy = []
f1_score_dummy = []

accuracy_nb = []
precision_nb = []
recall_nb = []
f1_score_nb = []


X, y = pred_patients_demographics_df.iloc[:, 1:-1], pred_patients_demographics_df.iloc[:, [-1]].values

X.info()
X.head()

n_splits = 5
# model = OneVsRestClassifier(DecisionTreeClassifier(class_weight="balanced"))
model = DecisionTreeClassifier(class_weight="balanced", min_samples_split=50, min_samples_leaf=50)

# dummy_model = OneVsRestClassifier(DummyClassifier(strategy="most_frequent"))
dummy_model = DummyClassifier(strategy="most_frequent")

nb_model = GaussianNB()

skf = StratifiedKFold(n_splits=n_splits, random_state=0, shuffle=True)

for train_index, test_index in skf.split(X, y):
    # Separa em variáveis de treino e teste
    x_train, y_train = X.values[train_index], y[train_index]
    x_test, y_test = X.values[test_index], y[test_index]

    # Fit do classificador c
    fitted_model = model.fit(x_train, y_train.ravel())
    fitted_dummy = dummy_model.fit(x_train, y_train.ravel())
    fitted_nb = nb_model.fit(x_train, y_train.ravel())

    # Predição dos valores de teste e valor de acurácia
    y_pred = fitted_model.predict(x_test)
    y_dummy = fitted_dummy.predict(x_test)
    y_nb = fitted_nb.predict(x_test)
    
    accuracy_tree.append(accuracy_score(y_test, y_pred))
    precision_tree.append(precision_score(y_test, y_pred, average='weighted'))
    recall_tree.append(recall_score(y_test, y_pred, average='weighted'))
    f1_score_tree.append(f1_score(y_test, y_pred, average='weighted'))

    accuracy_dummy.append(accuracy_score(y_test, y_dummy))
    precision_dummy.append(precision_score(y_test, y_dummy, average='weighted'))
    recall_dummy.append(recall_score(y_test, y_dummy, average='weighted'))
    f1_score_dummy.append(f1_score(y_test, y_dummy, average='weighted'))

    accuracy_nb.append(accuracy_score(y_test, y_nb))
    precision_nb.append(precision_score(y_test, y_nb, average='weighted'))
    recall_nb.append(recall_score(y_test, y_nb, average='weighted'))
    f1_score_nb.append(f1_score(y_test, y_nb, average='weighted'))


#     print(balanced_accuracy_score(y_test, y_pred))
#     print(accuracy_score(y_test, y_pred))
#     print(precision_score(y_test, y_pred, average='weighted'))
#     print(recall_score(y_test, y_pred, average='weighted'))
#     print(f1_score(y_test, y_pred, average='weighted'))
#     print(balanced_accuracy_score(y_test, y_dummy))
#     print("")

# importance = model.feature_importances_
# for i,v in enumerate(zip(importance, pred_demographics_df[1:])):
#     print(i, v)

print(np.mean(accuracy_tree))
print(np.mean(precision_tree))
print(np.mean(recall_tree))
print(np.mean(f1_score_tree))
print("")
print(np.mean(accuracy_dummy))
print(np.mean(precision_dummy))
print(np.mean(recall_dummy))
print(np.mean(f1_score_dummy))
print("")
print(np.mean(accuracy_nb))
print(np.mean(precision_nb))
print(np.mean(recall_nb))
print(np.mean(f1_score_nb))

In [ ]:
X

In [ ]:
export_graphviz(model, out_file="272/272-icd_admissions_patients.dot", feature_names=list(X.columns), filled=True, rounded=True, impurity=False, class_names=model.classes_.astype(str))
